In [ ]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy import stats
from sklearn import linear_model
from sklearn.model_selection import train_test_split

import os
from glob import glob
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

np.set_printoptions(precision=2)
pd.set_option('display.max_rows', 200)

# Get id from MRI set

In [ ]:
df_mri_013 = pd.read_csv("df_013_MRI_features_25042022", sep=",", header=0)
id_mri_013 = df_mri_013["v1: id"]
id_013 =[int(i[:7]) for i in id_mri_013]

df_mri_135 = pd.read_csv("df_135_MRI_features_25042022", sep=",", header=0)
id_mri_135 = df_mri_135["v1: id"]
id_135 =[int(i[:7]) for i in id_mri_135]


df_mri_356 = pd.read_csv("df_356_MRI_features_25042022", sep=",", header=0)
id_mri_356 = df_mri_356["v1: id"]
id_356 =[int(i[:7]) for i in id_mri_356]

In [ ]:
df_clin = pd.read_csv("matched_15032022", sep=",", header=0)
df_clin= df_clin.set_index("Unnamed: 0")

# Hjælpefunktioner

In [ ]:
#calculate confidence interval
def get_standard_diviation(values):
  mean = np.mean(values)
  n = len(values)
  s = np.sqrt(np.sum((values-mean)**2)/(n-1))

  return s


def biniaryyn_variable(dataframe, variable):
       
    # binary labels
    dataframe[variable] = dataframe[variable].replace(['.: Missing Form/Incomplete Workbook','0: No','1: Yes'],[0.5,0,1])

    return dataframe
    
def pain_values(dataframe,variable):
    
    # Set missing values as mean        
    dataframe[variable] = dataframe[variable].replace(['0: None','1: Mild','2: Moderate','3: Severe','4: Extreme'],[0,1,2,3,4])
    
    mean = np.mean(dataframe[dataframe[variable]!='.: Missing Form/Incomplete Workbook'][variable])
    dataframe[variable] = dataframe[variable].replace(['.: Missing Form/Incomplete Workbook'],[mean])
    
    

def pain_values2(dataframe,variable):

    dataframe[variable] = dataframe[variable].replace(['0: No pain', '1: 1', '2: 2', '3: 3', '4: 4', '5: 5','6: 6', '7: 7', '8: 8', '9: 9','10: Pain as bad as you can imagine'],[0,1,2,3,4,5,6,7,8,9,10])
    
    mean = np.mean(dataframe[dataframe[variable]!='.: Missing Form/Incomplete Workbook'][variable])
    dataframe[variable] = dataframe[variable].replace(['.: Missing Form/Incomplete Workbook'],[mean])
    
def emotional_values(dataframe, variable):
    # Replace missing values with mean 
    
    dataframe[variable] = dataframe[variable].replace(['1: Rarely/none of the time (<1 day)','2: Some of the time (1-2 days)','3: Much of the time (3-4 days)','4: Most or all of the time (5-7 days)'],[1,2,3,4])
    
    mean = np.mean(dataframe[dataframe[variable]!='.: Missing Form/Incomplete Workbook'][variable])
    
    dataframe[variable] = dataframe[variable].replace(['.: Missing Form/Incomplete Workbook'],[mean])  

    
def lifechane_values(dataframe, variable):
        dataframe[variable] = dataframe[variable].replace(['.: Missing Form/Incomplete Workbook','0: Not at all','1: Mildly', '2: Moderately',  '3: Severely', '4: Totally'],[0,0,1,2,3,4])

def nan_to_mean(dataframe, variable):
        mean = np.mean(dataframe[variable])
        dataframe[variable] = dataframe[variable].replace(['.: Missing Form/Incomplete Workbook'],[mean])

def vitamins(dataframe, variable):
            dataframe[variable] = dataframe[variable].replace(['.: Missing Form/Incomplete Workbook',
                                                                 '1: Didn t take',
                                                                 '2: A few days per month',
                                                                 '3: 4-6 days per week',
                                                                 '4: Every day',
                                                                 '5: 1-3 days per week'],[2,0,1,3,5,4])

# Enroll

In [ ]:
enroll = "C:/Users/python_test/Desktop/Speciale/OAIdata21/OAIdata21/Enrollees.txt"
df_enroll = pd.read_csv(enroll, sep="|", header=0, index_col="ID")
df_enroll = df_enroll["P02SEX"].replace(['1: Male','2: Female'],[0,1])

visit00 = "C:/Users/python_test/Desktop/Speciale/OAIdata21/OAIdata21/AllClinical00.txt"
df_visit00 = pd.read_csv(visit00, sep="|", header=0, index_col="ID")

# choose which variables to use
    # V00AGE age
    # Lives with spouse
age = df_visit00[["V00AGE", "V00LIVE1","V00LIVENO"]]
age["V00LIVE1"] = age["V00LIVE1"].replace(['.: Missing Form/Incomplete Workbook','0: No','1: Yes'],[0.5,0,1])
age["V00LIVENO"] = age["V00LIVENO"].replace(['.: Missing Form/Incomplete Workbook'],["0"])
age["V00LIVENO"] = [int(i[:1]) for i in age["V00LIVENO"]]

# Get data

In [ ]:
def get_data(list_id, visits):
    path_0 =  "C:/Users/python_test/Desktop/Speciale/OAIdata21/OAIdata21/AllClinical"+"00"+".txt"
    dataframe_0 = pd.read_csv(path_0, sep="|", header=0, index_col="ID")
    dataframe_0 = dataframe_0[dataframe_0.index.isin(list_id)]
    
    dataframe_0=dataframe_0[["P01MOMKRCV","P01DADKRCV","P01BROKRCV","P01SISKRCV"]]

    biniaryyn_variable(dataframe_0,"P01MOMKRCV")
    biniaryyn_variable(dataframe_0,"P01DADKRCV")
    biniaryyn_variable(dataframe_0,"P01BROKRCV")
    biniaryyn_variable(dataframe_0,"P01SISKRCV")

    data = []
    
    for visit in visits: 
        
        path =  "C:/Users/python_test/Desktop/Speciale/OAIdata21/OAIdata21/AllClinical"+visit+".txt"
        dataframe = pd.read_csv(path, sep="|", header=0, index_col="ID")
        dataframe = dataframe[dataframe.index.isin(list_id)]
        visit= "V"+visit

        if visit in ["V00"]:
            variables_of_interest = [visit+"WOMADLL", visit+"WOMADLR", visit+"WOMKPL", visit+"WOMKPR",visit+ "WOMSTFL",visit+"WOMSTFR",#visit+"WOMTSL", visit+"WOMTSR",# visit+"PMLKRCV", visit+"PMRKRCV", # pain
                                   visit+"DILKN2",visit+"DILKN10",visit+"DILKN11",visit+"DIRKN2",visit+"DIRKN10",visit+"DIRKN11", visit+"KQOL2", #limitations
                                   visit+"PASE",
                                   "V00BONEFX", #fracture after 45 month
                                   "P01BMI", #BMI] # Exercise
                                   "V00FALL"]# Histiry
            

        if visit in ["V01","V03","V05","V06"]:
            variables_of_interest = [visit+"WOMADLL", visit+"WOMADLR", visit+"WOMKPL", visit+"WOMKPR",visit+ "WOMSTFL",visit+"WOMSTFR",#visit+"WOMTSL", visit+"WOMTSR",# visit+"PMLKRCV", visit+"PMRKRCV", # pain
                                   visit+"DILKN2",visit+"DILKN10",visit+"DILKN11",visit+"DIRKN2",visit+"DIRKN10",visit+"DIRKN11", visit+"KQOL2", #limitations
                                   visit+"PASE",
                                   visit+"BONFX", #fracture last month
                                   visit+"BMI",
                                   visit+"FALL"] #BMI] # Exercise

        dataframe = dataframe[variables_of_interest]
        biniaryyn_variable(dataframe,visit+"FALL")        

        if visit in ["V01","V03","V05","V06"]:
            biniaryyn_variable(dataframe,visit+"BONFX")

        if visit in ["V00"]:
            biniaryyn_variable(dataframe,"V00BONEFX")
            
        pain_values(dataframe,visit+"DILKN10")
        pain_values(dataframe,visit+"DILKN11")
        pain_values(dataframe,visit+"DILKN2")
        pain_values(dataframe,visit+"DIRKN10")
        pain_values(dataframe,visit+"DIRKN11")
        pain_values(dataframe,visit+"DIRKN2")

        lifechane_values(dataframe,visit+"KQOL2")

        #Set missing values to mean
        for col in dataframe.columns:
            mean = np.mean(dataframe[col])
            dataframe[col] = dataframe[col].fillna(mean)


        data.append(dataframe)
        
#For three visits
    if len(visits) ==3:
        return pd.concat([df_enroll,dataframe_0,age,data[0],data[1],data[2]],axis=1,join="inner")

# For multiple visits
    else:
        return pd.concat([df_enroll,dataframe_0,age,data[0],data[1],data[2],data[3], data[4]],axis=1,join="inner")

In [ ]:
df_013 = get_data(id_013, ["00", "01", "03"])
df_135 = get_data(id_135, ["01", "03", "05"])
df_356 = get_data(id_356, ["03", "05", "06"])
df_01356 = get_data(id_356, ["00","01","03", "05", "06"])


df_013.shape, df_135.shape, df_356.shape

In [ ]:
df_013["iD"]=df_013.index
df_135["iD"]=df_135.index
df_356["iD"]=df_356.index
df_01356["iD"] = df_01356.index

# Get labels

In [ ]:
#labels
label_path = "C:/Users/python_test/Desktop/Speciale/OAIdata21/OAIdata21/Outcomes99.txt"
df_labels = pd.read_csv(label_path, sep="|", header=0, index_col="id")
#set(df_labels[df_labels["V99ERKVSPR"]!=".: Missing Form/Incomplete Workbook"]["V99ERKVSPR"])

In [ ]:
def get_labels(dataframe):
    dataframe["y_L"]=0
    dataframe["y_R"]=0
    
    for id in dataframe.index:
        if df_labels.loc[id]["V99ELKTLPR"]!= '.: Missing Form/Incomplete Workbook':
            dataframe.loc[id,"y_L"] = 1            

        if df_labels.loc[id]["V99ERKTLPR"]!= '.: Missing Form/Incomplete Workbook':
            dataframe.loc[id,"y_R"] = 1 
            
    #dataframe["y"] =dataframe["y_L"]+dataframe["y_R"]
    
    return dataframe

df_013 = get_labels(df_013)
df_135 = get_labels(df_135)
df_356 = get_labels(df_356)
df_01356 = get_labels(df_01356)

In [ ]:
# Stack to one df
df = pd.DataFrame( np.vstack( [ np.array(df_013), np.array(df_135), np.array(df_356) ] ) )
df = df.set_axis(list(df_013.columns), axis=1)

In [ ]:
df_013.shape, df_135.shape, df_356.shape, df.shape, df_01356.shape

# Train and val split

In [ ]:
TRAIN = pd.read_csv("matched_15032022", sep=",", header=0)
TRAIN = TRAIN.set_index("Unnamed: 0")
TRAIN = TRAIN.index

VAL = pd.read_csv("validation_15032022", sep=",", header=0)
VAL = VAL.set_index("Unnamed: 0")
VAL = VAL.index

In [ ]:
#Split for 3 visits
print("All: ", df_013.shape )

# Select validation or training mode VAL or TRAIN
train = df.loc[ df['iD'].isin(TRAIN)]
val = df.loc[~df['iD'].isin(TRAIN)]

#train = train.drop(["iD"],axis=1)
#val = val.drop(["iD"],axis=1)

print("Train", train.shape)
print("Val", val.shape)
train.shape[0]+val.shape[0]

In [ ]:
#split for 5 visits
print("All: ", df_01356.shape )

# Select validation or training mode VAL or TRAIN
train_01356 = df_01356.loc[ df_01356['iD'].isin(TRAIN)]
val_01356 = df_01356.loc[~df_01356['iD'].isin(TRAIN)]

#train_01356 = train_01356.drop(["iD"],axis=1)
#val_01356 = val_01356.drop(["iD"],axis=1)

print("Train", train_01356.shape)
print("Val", val_01356.shape)
train_01356.shape[0]+val_01356.shape[0]

# Split left right

In [ ]:
# Training
# augment knees i.e split kneewise
df_L = train [['P02SEX', "V00LIVE1","V00LIVENO","P01MOMKRCV","P01DADKRCV", "P01BROKRCV","P01SISKRCV",
           'V00WOMADLL', 'V00WOMKPL', 'V00WOMSTFL',"V00FALL", "P01BMI",
           'V00DILKN2','V00DILKN10', 'V00DILKN11', 'V00KQOL2', 'V00PASE', 'V00BONEFX', 
           'V01WOMADLL', 'V01WOMKPL', 'V01WOMSTFL',"V01FALL","V01BMI",
           'V01DILKN2','V01DILKN10', 'V01DILKN11', 'V01KQOL2', 'V01PASE', 'V01BONFX', 
           'V03WOMADLL', 'V03WOMKPL', 'V03WOMSTFL',"V03FALL","V03BMI",
           'V03DILKN2','V03DILKN10', 'V03DILKN11', 'V03KQOL2', 'V03PASE', 'V03BONFX',  
           "y_L","iD"]]
          
df_R = train [[ 'P02SEX', "V00LIVE1","V00LIVENO","P01MOMKRCV","P01DADKRCV","P01BROKRCV","P01SISKRCV",
           'V00WOMADLR', 'V00WOMKPR', 'V00WOMSTFR',"V00FALL","P01BMI",
           'V00DIRKN2','V00DIRKN10','V00DIRKN11','V00KQOL2','V00PASE','V00BONEFX',
           'V01WOMADLR', 'V01WOMKPR', 'V01WOMSTFR',"V01FALL","V01BMI" ,          
            'V01DIRKN2','V01DIRKN10','V01DIRKN11','V01KQOL2','V01PASE','V01BONFX', 
            'V03WOMADLR', 'V03WOMKPR', 'V03WOMSTFR',"V03FALL","V03BMI",
            'V03DIRKN2','V03DIRKN10','V03DIRKN11','V03KQOL2','V03PASE','V03BONFX',
            "y_R","iD"]]

#Validation
df_L_val = val [['P02SEX', "V00LIVE1","V00LIVENO","P01MOMKRCV","P01DADKRCV", "P01BROKRCV","P01SISKRCV",
           'V00WOMADLL', 'V00WOMKPL', 'V00WOMSTFL',"V00FALL", "P01BMI",
           'V00DILKN2','V00DILKN10', 'V00DILKN11', 'V00KQOL2', 'V00PASE', 'V00BONEFX', 
           'V01WOMADLL', 'V01WOMKPL', 'V01WOMSTFL',"V01FALL","V01BMI",
           'V01DILKN2','V01DILKN10', 'V01DILKN11', 'V01KQOL2', 'V01PASE', 'V01BONFX', 
           'V03WOMADLL', 'V03WOMKPL', 'V03WOMSTFL',"V03FALL","V03BMI",
           'V03DILKN2','V03DILKN10', 'V03DILKN11', 'V03KQOL2', 'V03PASE', 'V03BONFX',  
           "y_L","iD"]]
          
df_R_val = val [[ 'P02SEX', "V00LIVE1","V00LIVENO","P01MOMKRCV","P01DADKRCV","P01BROKRCV","P01SISKRCV",
           'V00WOMADLR', 'V00WOMKPR', 'V00WOMSTFR',"V00FALL","P01BMI",
           'V00DIRKN2','V00DIRKN10','V00DIRKN11','V00KQOL2','V00PASE','V00BONEFX',
           'V01WOMADLR', 'V01WOMKPR', 'V01WOMSTFR',"V01FALL","V01BMI" ,          
            'V01DIRKN2','V01DIRKN10','V01DIRKN11','V01KQOL2','V01PASE','V01BONFX', 
            'V03WOMADLR', 'V03WOMKPR', 'V03WOMSTFR',"V03FALL","V03BMI",
            'V03DIRKN2','V03DIRKN10','V03DIRKN11','V03KQOL2','V03PASE','V03BONFX',
            "y_R","iD"]]

In [ ]:
# for multiple visits 
# augment knees i.e split kneewise
df_01356_L = train_01356[['P02SEX', "V00LIVE1","V00LIVENO","P01MOMKRCV","P01DADKRCV", "P01BROKRCV","P01SISKRCV",
           'V00WOMADLL', 'V00WOMKPL', 'V00WOMSTFL',"V00FALL", "P01BMI",
           'V00DILKN2','V00DILKN10', 'V00DILKN11', 'V00KQOL2', 'V00PASE', 'V00BONEFX', 
           'V01WOMADLL', 'V01WOMKPL', 'V01WOMSTFL',"V01FALL","V01BMI",
           'V01DILKN2','V01DILKN10', 'V01DILKN11', 'V01KQOL2', 'V01PASE', 'V01BONFX', 
           'V03WOMADLL', 'V03WOMKPL', 'V03WOMSTFL',"V03FALL","V03BMI",
           'V03DILKN2','V03DILKN10', 'V03DILKN11', 'V03KQOL2', 'V03PASE', 'V03BONFX',
           'V05WOMADLL', 'V05WOMKPL', 'V05WOMSTFL',"V05FALL","V05BMI",
           'V05DILKN2','V05DILKN10', 'V05DILKN11', 'V05KQOL2', 'V05PASE', 'V05BONFX',  
           'V06WOMADLL', 'V06WOMKPL', 'V06WOMSTFL',"V06FALL","V06BMI",
           'V06DILKN2','V06DILKN10', 'V06DILKN11', 'V06KQOL2', 'V06PASE', 'V06BONFX',                         
           "y_L","iD"]]
          
df_01356_R = train_01356[[ 'P02SEX', "V00LIVE1","V00LIVENO","P01MOMKRCV","P01DADKRCV","P01BROKRCV","P01SISKRCV",
           'V00WOMADLR', 'V00WOMKPR', 'V00WOMSTFR',"V00FALL","P01BMI",
           'V00DIRKN2','V00DIRKN10','V00DIRKN11','V00KQOL2','V00PASE','V00BONEFX',
           'V01WOMADLR', 'V01WOMKPR', 'V01WOMSTFR',"V01FALL","V01BMI" ,          
            'V01DIRKN2','V01DIRKN10','V01DIRKN11','V01KQOL2','V01PASE','V01BONFX', 
            'V03WOMADLR', 'V03WOMKPR', 'V03WOMSTFR',"V03FALL","V03BMI",
            'V03DIRKN2','V03DIRKN10','V03DIRKN11','V03KQOL2','V03PASE','V03BONFX',
            'V05WOMADLR', 'V05WOMKPR', 'V05WOMSTFR',"V05FALL","V05BMI",
            'V05DIRKN2','V05DIRKN10','V05DIRKN11','V05KQOL2','V05PASE','V05BONFX',                       
            'V06WOMADLR', 'V06WOMKPR', 'V06WOMSTFR',"V06FALL","V06BMI",
            'V06DIRKN2','V06DIRKN10','V06DIRKN11','V06KQOL2','V06PASE','V06BONFX',            
            "y_R","iD"]]

# augment knees i.e split kneewise
df_01356_L_val = val_01356[['P02SEX', "V00LIVE1","V00LIVENO","P01MOMKRCV","P01DADKRCV", "P01BROKRCV","P01SISKRCV",
           'V00WOMADLL', 'V00WOMKPL', 'V00WOMSTFL',"V00FALL", "P01BMI",
           'V00DILKN2','V00DILKN10', 'V00DILKN11', 'V00KQOL2', 'V00PASE', 'V00BONEFX', 
           'V01WOMADLL', 'V01WOMKPL', 'V01WOMSTFL',"V01FALL","V01BMI",
           'V01DILKN2','V01DILKN10', 'V01DILKN11', 'V01KQOL2', 'V01PASE', 'V01BONFX', 
           'V03WOMADLL', 'V03WOMKPL', 'V03WOMSTFL',"V03FALL","V03BMI",
           'V03DILKN2','V03DILKN10', 'V03DILKN11', 'V03KQOL2', 'V03PASE', 'V03BONFX',
           'V05WOMADLL', 'V05WOMKPL', 'V05WOMSTFL',"V05FALL","V05BMI",
           'V05DILKN2','V05DILKN10', 'V05DILKN11', 'V05KQOL2', 'V05PASE', 'V05BONFX',  
           'V06WOMADLL', 'V06WOMKPL', 'V06WOMSTFL',"V06FALL","V06BMI",
           'V06DILKN2','V06DILKN10', 'V06DILKN11', 'V06KQOL2', 'V06PASE', 'V06BONFX',                         
           "y_L","iD"]]
          
df_01356_R_val = val_01356[[ 'P02SEX', "V00LIVE1","V00LIVENO","P01MOMKRCV","P01DADKRCV","P01BROKRCV","P01SISKRCV",
           'V00WOMADLR', 'V00WOMKPR', 'V00WOMSTFR',"V00FALL","P01BMI",
           'V00DIRKN2','V00DIRKN10','V00DIRKN11','V00KQOL2','V00PASE','V00BONEFX',
           'V01WOMADLR', 'V01WOMKPR', 'V01WOMSTFR',"V01FALL","V01BMI" ,          
            'V01DIRKN2','V01DIRKN10','V01DIRKN11','V01KQOL2','V01PASE','V01BONFX', 
            'V03WOMADLR', 'V03WOMKPR', 'V03WOMSTFR',"V03FALL","V03BMI",
            'V03DIRKN2','V03DIRKN10','V03DIRKN11','V03KQOL2','V03PASE','V03BONFX',
            'V05WOMADLR', 'V05WOMKPR', 'V05WOMSTFR',"V05FALL","V05BMI",
            'V05DIRKN2','V05DIRKN10','V05DIRKN11','V05KQOL2','V05PASE','V05BONFX',                       
            'V06WOMADLR', 'V06WOMKPR', 'V06WOMSTFR',"V06FALL","V06BMI",
            'V06DIRKN2','V06DIRKN10','V06DIRKN11','V06KQOL2','V06PASE','V06BONFX',            
            "y_R","iD"]]

In [ ]:
#Mult visits,  make data set 
lab_L_01356= df_01356_L["iD"].astype(int).astype(str)+"_Left"
lab_R_01356 = df_01356_R["iD"].astype(int).astype(str)+"_Right"
df_01356_L["iD"] = lab_L_01356
df_01356_R["iD"] = lab_R_01356

df_both_01356 = pd.DataFrame( np.vstack( [ np.array(df_01356_L), np.array(df_01356_R) ] ) )
df_both_01356 = df_both_01356.set_axis(list(df_01356_R.columns), axis=1)

# change to floats
for i in df_both_01356.columns:
    if i != "iD":
        df_both_01356[i] = df_both_01356[i].astype(float)
        
        
# also for validation set
lab_L_01356_val = df_01356_L_val["iD"].astype(int).astype(str)+"_Left"
lab_R_01356_val = df_01356_R_val["iD"].astype(int).astype(str)+"_Right"
df_01356_L_val["iD"] = lab_L_01356_val
df_01356_R_val["iD"] = lab_R_01356_val

df_both_01356_val = pd.DataFrame( np.vstack( [ np.array(df_01356_L_val), np.array(df_01356_R_val) ] ) )
df_both_01356_val = df_both_01356_val.set_axis(list(df_01356_R_val.columns), axis=1)

# change to floats
for i in df_both_01356_val.columns:
    if i != "iD":
        df_both_01356_val[i] = df_both_01356_val[i].astype(float)

In [ ]:
# Rename the variables
df_both_01356 = df_both_01356.rename(columns={"P02SEX": "Sex (V0)", 
                        "V00AGE": "Age (V1)",
                        "V00LIVE1": "Lives with a Spouse(V0)",
                        "V00LIVENO": "Other People in Household",
                        "P01MOMKRCV":"Mom had TKR-surgery",
                        "P01DADKRCV":"Dad had TKR-surgery",
                        "P01BROKRCV":"Brother had TKR-surgery",
                        "P01SISKRCV":"Sister had TKR-surgery",
 
                        'P01BMI': "BMI (V1)",
                        "V00FALL":"Fall within last 12 month (v1)",
                        'V00WOMADLR':"WOMAC Disability Score (calc) (v1)",
                        'V00WOMKPR': "WOMAC Pain Score (calc) (v1)", 
                        'V00WOMSTFR': "WOMAC Stiffness Score (calc) (v1)",
                        #"V00WOMTSR":"WOMAC (V1)",
                        "V00DIRKN2":"Upstairs, last 7 days (V1)", 
                        "V00DIRKN10":"Get out of bed, last 7 days (V1)",
                        "V00DIRKN11":"Socks off, last 7 days (V1)",
                        "V00KQOL2": "modified lifestyle (V1)",
                        "V00PASE":"Physical Activity Scale (V1)",
                        "V00BONEFX":"Broke or fractured bone (V1)",
                        
                        "V01BMI": "BMI (V2)",
                        "V01FALL":"Fall within last 12 month (v2)",
                        'V01WOMADLR':"WOMAC Disability Score (calc) (v2)",
                        'V01WOMKPR': "WOMAC Pain Score (calc) (v2)", 
                        'V01WOMSTFR': "WOMAC Stiffness Score (calc) (v2)",
                        #'V01WOMTSR':"WOMAC (V2)",
                        'V01DIRKN2':"Upstairs, last 7 days (V2)",
                        'V01DIRKN10':"Get out of bed, last 7 days (V2)", 
                        'V01DIRKN11': "Socks off, last 7 days (V2)", 
                        'V01KQOL2':"modified lifestyle (V2)", 
                        'V01PASE':"Physical Activity Scale (V2)" ,
                        'V01BONFX':"Broke or fractured bone (V2)",
                                  
                        'V03BMI':"BMI (V3)",
                        "V03FALL":"Fall within last 12 month (v3)",
                        'V03WOMADLR':"WOMAC Disability Score (calc) (v3)",
                        'V03WOMKPR': "WOMAC Pain Score (calc) (v3)", 
                        'V03WOMSTFR': "WOMAC Stiffness Score (calc) (v3)",
                        #'V03WOMTSR':"WOMAC (V3)",
                        'V03DIRKN2':"Upstairs, last 7 days (V3)", 
                        'V03DIRKN10':"Get out of bed, last 7 days (V3)",
                        'V03DIRKN11':"Socks off, last 7 days (V3)",
                        'V03KQOL2':"Modified lifestyle (V3)", 
                        'V03PASE':"Physical Activity Scale (V3)" ,
                        'V03BONFX':"Broke or fractured bone (V3)", 
                        
                        'V05BMI':"BMI (V5)",
                        "V05FALL":"Fall within last 12 month (v5)",
                        'V05WOMADLR':"WOMAC Disability Score (calc) (v5)",
                        'V05WOMKPR': "WOMAC Pain Score (calc) (v5)", 
                        'V05WOMSTFR': "WOMAC Stiffness Score (calc) (v5)",
                        #'V05WOMTSR':"WOMAC (V5)",
                        'V05DIRKN2':"Upstairs, last 7 days (V5)", 
                        'V05DIRKN10':"Get out of bed, last 7 days (V5)",
                        'V05DIRKN11':"Socks off, last 7 days (V5)",
                        'V05KQOL2':"Modified lifestyle (V5)", 
                        'V05PASE':"Physical Activity Scale (V5)" ,
                        'V05BONFX':"Broke or fractured bone (V5)", 
                        
                                                'V06BMI':"BMI (V6)",
                        "V06FALL":"Fall within last 12 month (v6)",
                        'V06WOMADLR':"WOMAC Disability Score (calc) (v6)",
                        'V06WOMKPR': "WOMAC Pain Score (calc) (v6)", 
                        'V06WOMSTFR': "WOMAC Stiffness Score (calc) (v6)",
                        #'V06WOMTSR':"WOMAC (V6)",
                        'V06DIRKN2':"Upstairs, last 7 days (V6)", 
                        'V06DIRKN10':"Get out of bed, last 7 days (V6)",
                        'V06DIRKN11':"Socks off, last 7 days (V6)",
                        'V06KQOL2':"Modified lifestyle (V6)", 
                        'V06PASE':"Physical Activity Scale (V6)" ,
                        'V06BONFX':"Broke or fractured bone (V6)", 
                                  
                        "y_R": "y"
                                               })

total = df_both_01356["Mom had TKR-surgery"]+df_both_01356["Dad had TKR-surgery"]+df_both_01356["Brother had TKR-surgery"]+df_both_01356["Sister had TKR-surgery"]
df_both_01356.insert(1, 'Total Family History', total)

# Rename the variables
df_both_01356_val = df_both_01356_val.rename(columns={"P02SEX": "Sex (V0)", 
                        "V00AGE": "Age (V1)",
                        "V00LIVE1": "Lives with a Spouse(V0)",
                        "V00LIVENO": "Other People in Household",
                        "P01MOMKRCV":"Mom had TKR-surgery",
                        "P01DADKRCV":"Dad had TKR-surgery",
                        "P01BROKRCV":"Brother had TKR-surgery",
                        "P01SISKRCV":"Sister had TKR-surgery",
 
                        'P01BMI': "BMI (V1)",
                        "V00FALL":"Fall within last 12 month (v1)",
                        'V00WOMADLR':"WOMAC Disability Score (calc) (v1)",
                        'V00WOMKPR': "WOMAC Pain Score (calc) (v1)", 
                        'V00WOMSTFR': "WOMAC Stiffness Score (calc) (v1)",
                        #"V00WOMTSR":"WOMAC (V1)",
                        "V00DIRKN2":"Upstairs, last 7 days (V1)", 
                        "V00DIRKN10":"Get out of bed, last 7 days (V1)",
                        "V00DIRKN11":"Socks off, last 7 days (V1)",
                        "V00KQOL2": "modified lifestyle (V1)",
                        "V00PASE":"Physical Activity Scale (V1)",
                        "V00BONEFX":"Broke or fractured bone (V1)",
                        
                        "V01BMI": "BMI (V2)",
                        "V01FALL":"Fall within last 12 month (v2)",
                        'V01WOMADLR':"WOMAC Disability Score (calc) (v2)",
                        'V01WOMKPR': "WOMAC Pain Score (calc) (v2)", 
                        'V01WOMSTFR': "WOMAC Stiffness Score (calc) (v2)",
                        #'V01WOMTSR':"WOMAC (V2)",
                        'V01DIRKN2':"Upstairs, last 7 days (V2)",
                        'V01DIRKN10':"Get out of bed, last 7 days (V2)", 
                        'V01DIRKN11': "Socks off, last 7 days (V2)", 
                        'V01KQOL2':"modified lifestyle (V2)", 
                        'V01PASE':"Physical Activity Scale (V2)" ,
                        'V01BONFX':"Broke or fractured bone (V2)",
                                  
                        'V03BMI':"BMI (V3)",
                        "V03FALL":"Fall within last 12 month (v3)",
                        'V03WOMADLR':"WOMAC Disability Score (calc) (v3)",
                        'V03WOMKPR': "WOMAC Pain Score (calc) (v3)", 
                        'V03WOMSTFR': "WOMAC Stiffness Score (calc) (v3)",
                        #'V03WOMTSR':"WOMAC (V3)",
                        'V03DIRKN2':"Upstairs, last 7 days (V3)", 
                        'V03DIRKN10':"Get out of bed, last 7 days (V3)",
                        'V03DIRKN11':"Socks off, last 7 days (V3)",
                        'V03KQOL2':"Modified lifestyle (V3)", 
                        'V03PASE':"Physical Activity Scale (V3)" ,
                        'V03BONFX':"Broke or fractured bone (V3)", 
                        
                        'V05BMI':"BMI (V5)",
                        "V05FALL":"Fall within last 12 month (v5)",
                        'V05WOMADLR':"WOMAC Disability Score (calc) (v5)",
                        'V05WOMKPR': "WOMAC Pain Score (calc) (v5)", 
                        'V05WOMSTFR': "WOMAC Stiffness Score (calc) (v5)",
                        #'V05WOMTSR':"WOMAC (V5)",
                        'V05DIRKN2':"Upstairs, last 7 days (V5)", 
                        'V05DIRKN10':"Get out of bed, last 7 days (V5)",
                        'V05DIRKN11':"Socks off, last 7 days (V5)",
                        'V05KQOL2':"Modified lifestyle (V5)", 
                        'V05PASE':"Physical Activity Scale (V5)" ,
                        'V05BONFX':"Broke or fractured bone (V5)", 
                        
                                                'V06BMI':"BMI (V6)",
                        "V06FALL":"Fall within last 12 month (v6)",
                        'V06WOMADLR':"WOMAC Disability Score (calc) (v6)",
                        'V06WOMKPR': "WOMAC Pain Score (calc) (v6)", 
                        'V06WOMSTFR': "WOMAC Stiffness Score (calc) (v6)",
                        #'V06WOMTSR':"WOMAC (V6)",
                        'V06DIRKN2':"Upstairs, last 7 days (V6)", 
                        'V06DIRKN10':"Get out of bed, last 7 days (V6)",
                        'V06DIRKN11':"Socks off, last 7 days (V6)",
                        'V06KQOL2':"Modified lifestyle (V6)", 
                        'V06PASE':"Physical Activity Scale (V6)" ,
                        'V06BONFX':"Broke or fractured bone (V6)", 
                                  
                        "y_R": "y"
                                               })

total_val = df_both_01356_val["Mom had TKR-surgery"]+df_both_01356_val["Dad had TKR-surgery"]+df_both_01356_val["Brother had TKR-surgery"]+df_both_01356_val["Sister had TKR-surgery"]
df_both_01356_val.insert(1, 'Total Family History', total_val)

In [ ]:
# back to three visits
lab_L= df_L["iD"].astype(int).astype(str)+"_Left"
lab_R = df_R["iD"].astype(int).astype(str)+"_Right"
df_L["iD"] = lab_L
df_R["iD"] = lab_R

lab_L_val = df_L_val["iD"].astype(int).astype(str)+"_Left"
lab_R_val = df_R_val["iD"].astype(int).astype(str)+"_Right"
df_L_val["iD"] = lab_L_val
df_R_val["iD"] = lab_R_val

In [ ]:
df_both = pd.DataFrame( np.vstack( [ np.array(df_L), np.array(df_R) ] ) )
df_both = df_both.set_axis(list(df_R.columns), axis=1)

# change to floats
for i in df_both.columns:
    if i != "iD":
        df_both[i] = df_both[i].astype(float)
        
df_both_val = pd.DataFrame( np.vstack( [ np.array(df_L_val), np.array(df_R_val) ] ) )
df_both_val = df_both_val.set_axis(list(df_R_val.columns), axis=1)

# change to floats
for i in df_both_val.columns:
    if i != "iD":
        df_both_val[i] = df_both_val[i].astype(float)

In [ ]:
# Rename the variables
df_both = df_both.rename(columns={"P02SEX": "Sex (V0)", 
                        "V00AGE": "Age (V1)",
                        "V00LIVE1": "Lives with a Spouse(V0)",
                        "V00LIVENO": "Other People in Household",
                        "P01MOMKRCV":"Mom had TKR-surgery",
                        "P01DADKRCV":"Dad had TKR-surgery",
                        "P01BROKRCV":"Brother had TKR-surgery",
                        "P01SISKRCV":"Sister had TKR-surgery",
 
                        'P01BMI': "BMI (V1)",
                        "V00FALL":"Fall within last 12 month (v1)",
                        'V00WOMADLR':"WOMAC Disability Score (calc) (v1)",
                        'V00WOMKPR': "WOMAC Pain Score (calc) (v1)", 
                        'V00WOMSTFR': "WOMAC Stiffness Score (calc) (v1)",
                        #"V00WOMTSR":"WOMAC (V1)",
                        "V00DIRKN2":"Upstairs, last 7 days (V1)", 
                        "V00DIRKN10":"Get out of bed, last 7 days (V1)",
                        "V00DIRKN11":"Socks off, last 7 days (V1)",
                        "V00KQOL2": "modified lifestyle (V1)",
                        "V00PASE":"Physical Activity Scale (V1)",
                        "V00BONEFX":"Broke or fractured bone (V1)",
                        
                        "V01BMI": "BMI (V2)",
                        "V01FALL":"Fall within last 12 month (v2)",
                        'V01WOMADLR':"WOMAC Disability Score (calc) (v2)",
                        'V01WOMKPR': "WOMAC Pain Score (calc) (v2)", 
                        'V01WOMSTFR': "WOMAC Stiffness Score (calc) (v2)",
                        #'V01WOMTSR':"WOMAC (V2)",
                        'V01DIRKN2':"Upstairs, last 7 days (V2)",
                        'V01DIRKN10':"Get out of bed, last 7 days (V2)", 
                        'V01DIRKN11': "Socks off, last 7 days (V2)", 
                        'V01KQOL2':"modified lifestyle (V2)", 
                        'V01PASE':"Physical Activity Scale (V2)" ,
                        'V01BONFX':"Broke or fractured bone (V2)",
                                  
                        'V03BMI':"BMI (V3)",
                        "V03FALL":"Fall within last 12 month (v3)",
                        'V03WOMADLR':"WOMAC Disability Score (calc) (v3)",
                        'V03WOMKPR': "WOMAC Pain Score (calc) (v3)", 
                        'V03WOMSTFR': "WOMAC Stiffness Score (calc) (v3)",
                        #'V03WOMTSR':"WOMAC (V3)",
                        'V03DIRKN2':"Upstairs, last 7 days (V3)", 
                        'V03DIRKN10':"Get out of bed, last 7 days (V3)",
                        'V03DIRKN11':"Socks off, last 7 days (V3)",
                        'V03KQOL2':"Modified lifestyle (V3)", 
                        'V03PASE':"Physical Activity Scale (V3)" ,
                        'V03BONFX':"Broke or fractured bone (V3)", 
                                  
                        "y_R": "y"
                                               })

# Rename the variables
df_both_val = df_both_val.rename(columns={"P02SEX": "Sex (V0)", 
                        "V00AGE": "Age (V1)",
                        "V00LIVE1": "Lives with a Spouse(V0)",
                        "V00LIVENO": "Other People in Household",
                        "P01MOMKRCV":"Mom had TKR-surgery",
                        "P01DADKRCV":"Dad had TKR-surgery",
                        "P01BROKRCV":"Brother had TKR-surgery",
                        "P01SISKRCV":"Sister had TKR-surgery",
 
                        'P01BMI': "BMI (V1)",
                        "V00FALL":"Fall within last 12 month (v1)",
                        'V00WOMADLR':"WOMAC Disability Score (calc) (v1)",
                        'V00WOMKPR': "WOMAC Pain Score (calc) (v1)", 
                        'V00WOMSTFR': "WOMAC Stiffness Score (calc) (v1)",
                        #"V00WOMTSR":"WOMAC (V1)",
                        "V00DIRKN2":"Upstairs, last 7 days (V1)", 
                        "V00DIRKN10":"Get out of bed, last 7 days (V1)",
                        "V00DIRKN11":"Socks off, last 7 days (V1)",
                        "V00KQOL2": "modified lifestyle (V1)",
                        "V00PASE":"Physical Activity Scale (V1)",
                        "V00BONEFX":"Broke or fractured bone (V1)",
                        
                        "V01BMI": "BMI (V2)",
                        "V01FALL":"Fall within last 12 month (v2)",
                        'V01WOMADLR':"WOMAC Disability Score (calc) (v2)",
                        'V01WOMKPR': "WOMAC Pain Score (calc) (v2)", 
                        'V01WOMSTFR': "WOMAC Stiffness Score (calc) (v2)",
                        #'V01WOMTSR':"WOMAC (V2)",
                        'V01DIRKN2':"Upstairs, last 7 days (V2)",
                        'V01DIRKN10':"Get out of bed, last 7 days (V2)", 
                        'V01DIRKN11': "Socks off, last 7 days (V2)", 
                        'V01KQOL2':"modified lifestyle (V2)", 
                        'V01PASE':"Physical Activity Scale (V2)" ,
                        'V01BONFX':"Broke or fractured bone (V2)",
                                  
                        'V03BMI':"BMI (V3)",
                        "V03FALL":"Fall within last 12 month (v3)",
                        'V03WOMADLR':"WOMAC Disability Score (calc) (v3)",
                        'V03WOMKPR': "WOMAC Pain Score (calc) (v3)", 
                        'V03WOMSTFR': "WOMAC Stiffness Score (calc) (v3)",
                        #'V03WOMTSR':"WOMAC (V3)",
                        'V03DIRKN2':"Upstairs, last 7 days (V3)", 
                        'V03DIRKN10':"Get out of bed, last 7 days (V3)",
                        'V03DIRKN11':"Socks off, last 7 days (V3)",
                        'V03KQOL2':"Modified lifestyle (V3)", 
                        'V03PASE':"Physical Activity Scale (V3)" ,
                        'V03BONFX':"Broke or fractured bone (V3)", 
                                  
                        "y_R": "y"
                                               })

In [ ]:
total = df_both["Mom had TKR-surgery"]+df_both["Dad had TKR-surgery"]+df_both["Brother had TKR-surgery"]+df_both["Sister had TKR-surgery"]
df_both.insert(1, 'Total Family History', total)

total_val = df_both_val["Mom had TKR-surgery"]+df_both_val["Dad had TKR-surgery"]+df_both_val["Brother had TKR-surgery"]+df_both_val["Sister had TKR-surgery"]
df_both_val.insert(1, 'Total Family History', total_val)

In [ ]:
#Save file
df_both.to_csv("clindata_new_25042022")

#Split to X y 
y = df_both["y"]
X = np.array(df_both.drop(["y","iD"],axis=1))

#Split to X y 
y_val= df_both_val["y"]
X_val = np.array(df_both_val.drop(["y","iD"],axis=1))

In [ ]:
#multiple visits
df_both_01356.to_csv("clindata_multiple_visits.csv")


#Split to X y 
y_01356 = df_both_01356["y"]
X_01356 = np.array(df_both_01356.drop(["y","iD"],axis=1))

y_01356_val = df_both_01356_val["y"]
X_01356_val = np.array(df_both_01356_val.drop(["y","iD"],axis=1))

# Make correlation matrix

In [ ]:
corr = df_both.iloc[:, :-1].corr(method='pearson')

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
fig, ax = plt.subplots(figsize=(15, 10))
sns.heatmap(corr, mask=mask, cmap="RdBu", vmin=-1, vmax=1, center=0, linewidths=.5)

fig.suptitle('Correlation matrix of Clinical Features', fontsize=15)
ax.text(0.77, 0.2, 'aegis4048.github.io', fontsize=13, ha='center', va='center',
         transform=ax.transAxes, color='grey', alpha=0.7)

fig.tight_layout()

In [ ]:
df_v5_3v = np.array(df_both_01356.iloc[:,:df_both.shape[1]-2])

# Random Forest

In [ ]:
# make stratified KFOLD 
n_split=10

skf = StratifiedKFold(n_splits=n_split, shuffle=False)
mean = 0
data =[]
preds_stat=np.array([])

for train_index, test_index in skf.split(X_01356, y_01356): #or X, y for three visits _01356. v5_3v
    Xtrain = X_01356[train_index]
    ytrain = y_01356[train_index]
    Xtest = X_01356[test_index]
    ytest = y_01356[test_index]
    
    #normalize data 
    m = np.mean(Xtrain,axis=0)
    std = np.std(Xtrain, axis =0)
    Xtrain = np.array((Xtrain - m )/std)
    Xtest = np.array((Xtest - m )/std)
    
    reg = RandomForestClassifier(max_depth=20).fit(Xtrain, ytrain)
    ypred = reg.predict(Xtest)
    #ypred = np.where(ypred<0,0, ypred)

    preds_stat = np.concatenate([preds_stat,ypred])


    print("Accuracy Score: ", roc_auc_score(ytest.astype(float), ypred))
    #print("Accuracy Score!: ",accuracy_score(ytest, ypred.round()))
    mean += roc_auc_score(ytest.astype(float), ypred)
    data.append(roc_auc_score(ytest.astype(float), ypred))
    
    
print("Mean accuracy score: ", mean/n_split)
print("SD: ", get_standard_diviation(data) )
print("All: ", data)

In [ ]:
#np.savetxt('RF_cli_3v.csv', preds_stat, delimiter=',')   
#np.savetxt('y_cli_3v.csv', y, delimiter=',')   
#np.savetxt('RF_cli_5v.csv', preds_stat, delimiter=',')  
#np.savetxt('y_cli_5v.csv', y_01356, delimiter=',')   
#np.savetxt('RF_cli_5v_3v.csv', preds_stat, delimiter=',')   
#np.savetxt('y_cli_5v_3v.csv', y_01356, delimiter=',')   


In [ ]:
preds_stat.shape

### Validation

In [ ]:
m = np.mean(X,axis=0)
std = np.std(X, axis =0)
X = (X - m )/std
X_val = (X_val - m )/std

reg_val = RandomForestClassifier(max_depth=20).fit(X, y)
ypred = reg_val.predict(X_val)
print("Accuracy Score: ", roc_auc_score(y_val.astype(float), ypred))

# Linear Regression

In [ ]:
# make stratified KFOLD 
n_split=10

skf = StratifiedKFold(n_splits=n_split, shuffle=False)
mean = 0
data =[]
preds_stat=np.array([])

for train_index, test_index in skf.split(X, y): #or X,y for 3 visits  y_01356, df_v5_3v
    Xtrain = X[train_index]
    ytrain = y[train_index]
    Xtest = X[test_index]
    ytest = y[test_index]
    
    #normalize data 
    m = np.mean(Xtrain,axis=0)
    std = np.std(Xtrain, axis =0)
    Xtrain = np.array((Xtrain - m )/std)
    Xtest = np.array((Xtest - m )/std)
    
    reg = LinearRegression().fit(Xtrain, ytrain)
    ypred = reg.predict(Xtest)
    ypred = np.where(ypred<0,0, ypred)

    preds_stat = np.concatenate([preds_stat,ypred])

    print("Accuracy Score: ", roc_auc_score(ytest.astype(float), ypred))
    #print("Accuracy Score!: ",accuracy_score(ytest, ypred.round()))
    mean += roc_auc_score(ytest.astype(float), ypred)
    data.append(roc_auc_score(ytest.astype(float), ypred))
    
    
print("Mean accuracy score: ", mean/n_split)
print("SSE: ", get_standard_diviation(data) )
print("All: ", data)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix( ytest,ypred.round())

In [ ]:
#np.savetxt('LinR_cli_3v.csv', preds_stat, delimiter=',')   
#np.savetxt('y_cli_3v.csv', y, delimiter=',')   
#np.savetxt('LinR_cli_5v.csv', preds_stat, delimiter=',')  
#np.savetxt('y_cli_5v.csv', y_01356, delimiter=',') 
#np.savetxt('LinR_cli_5v_3v.csv', preds_stat, delimiter=',')  

### Make validation

In [ ]:
m = np.mean(X,axis=0)
std = np.std(X, axis =0)
X = (X - m )/std
X_val = (X_val - m )/std

reg = LinearRegression()
reg = reg.fit(X_01356, y_01356)
ypred = reg.predict(X_01356_val)
#ypred = np.where(ypred<0,0, ypred)

print("Accuracy Score: ", roc_auc_score(y_01356_val.astype(float), ypred))

### One big linear regression

In [ ]:
#One Big Model
Xtrain, Xtest, ytrain, ytest = train_test_split(X,y, test_size=0.20, random_state=42, shuffle=True)

m = np.mean(Xtrain,axis=0)
std = np.std(Xtrain, axis =0)
Xtrain_pd = (Xtrain - m )/std

Xtrain = np.array((Xtrain - m )/std)
Xtest_pd = (Xtest - m )/std
Xtest = np.array((Xtest - m )/std)

reg = LinearRegression().fit(Xtrain, ytrain)
ypred = reg.predict(Xtest)
ypred = np.where(ypred<0,0, ypred)
    
print("Accuracy Score: ", roc_auc_score(ytest, ypred))
#print("Coefficients: \n", reg.coef_)
print("incept", reg.intercept_)

### T-statistics

In [ ]:
# Get corefficients
params = reg.coef_

# Add row to constants
newX = pd.DataFrame({"Constant":np.ones(len(Xtest))}).join(pd.DataFrame(Xtest))
column_names =["incept"]+list(df_both.columns[:-1])

# #calculate std 
m = np.mean(newX, axis=0)
n = len(newX)
std = np.sqrt(np.sum((newX-m)**2, axis=0)/(n))

#Add incepts
params = np.append(reg.intercept_,params)

t_values = params/std

# set se of incept to 0
std[0] = 0
std=np.array(std)
t_values[0] = 0
t_values = np.array(t_values)

myDF3 = pd.DataFrame()
myDF3["Coefficients"]= params
myDF3["Standard Errors"] =std
myDF3["t values"] = t_values
names=["incept"]+list(df_both.columns[:-1])
myDF3=myDF3.set_index(pd.Index(names))
myDF3

In [ ]:
#PlAY
k = 8

importance = myDF3["t values"][1:]
n=int((len(importance)-k)/3)#round((len(importance))/2)
# Get variables, connect to weights (#Note remove id and labels)
variables =[i[:-4] for i in myDF3.index[1:]]

# plot histograms
plt.figure(figsize=(30, 10))
plt.bar([x+k for x in range(len(importance[k+1:n+k]))], importance[k+1:n+k], alpha=0.5, color="b",label="V1",edgecolor=None)
plt.bar([x+k for x in range(len(importance[n+k+1:2*n+k]))], importance[n+k+1:2*n+k], alpha=0.5, color="g",label="V2",edgecolor=None)
plt.bar([x+k for x in range(len(importance[2*n+k:]))], importance[2*n+k:], alpha=0.5, color="r",label="V3",edgecolor=None)

plt.bar([x for x in range(len(importance[1:k-1]))], importance[1:k-1], color="black",label="Static Variables",edgecolor=None)

plt.xticks([x for x in range(len(importance[:n+k]))], variables[:n+k], rotation =90)
plt.plot(range(n+k+1), np.zeros(n+k+1), c="black")
#plt.yscale('symlog')
plt.grid(True, linestyle='-.')
#plt.ylim([-1,1])
plt.tick_params(labelcolor='r', labelsize=20, width=3)
plt.savefig("allfeat")
plt.legend(loc="best",fontsize="xx-large")
plt.show()

# Logistic regression

In [ ]:
# make stratified KFOLD 
mean_list=[]

mean = 0
for train_index, test_index in skf.split(X, y):
    Xtrain = X[train_index]
    ytrain = y[train_index]
    Xtest = X[test_index]
    ytest = y[test_index]
    
    m = np.mean(Xtrain,axis=0)
    std = np.std(Xtrain, axis =0)
    Xtrain = (Xtrain - m )/std
    Xtest = (Xtest - m )/std
    
    Log_reg = LogisticRegression().fit(Xtrain, ytrain) #max_iter=20000
    ypred = Log_reg.predict(Xtest)
    y_pred = Log_reg.predict_proba(Xtest)

    print("Accuracy Score: ", roc_auc_score(ytest.astype(float), y_pred[:,1]))
    mean +=roc_auc_score(ytest.astype(float), y_pred[:,1])
    mean_list.append(roc_auc_score(ytest.astype(float), y_pred[:,1]))
    
print("Mean accuracy score: ", mean/n_split)    
print("Confidence Interval: ", get_standard_diviation(mean_list))
print("ALL: ", mean_list)

### Maske Validation

In [ ]:
reg = LogisticRegression(max_iter=20000)
reg = reg.fit(X, y)
ypred = reg.predict(X_val)

print("Accuracy Score: ", roc_auc_score(y_val.astype(float), ypred))

## Lasso

In [ ]:
# make stratified KFOLD 
mean_list =[]

mean = 0
for train_index, test_index in skf.split(X, y):
    Xtrain = X[train_index]
    ytrain = y[train_index]
    Xtest = X[test_index]
    ytest = y[test_index]

    m = np.mean(Xtrain,axis=0)
    std = np.std(Xtrain, axis =0)
    Xtrain = (Xtrain - m )/std
    Xtest = (Xtest - m )/std

    reg_las = linear_model.Lasso(0.01)
    reg_las.fit(Xtrain, ytrain) 
    ypred = reg_las.predict(Xtest)

    print("Accuracy Score: ", roc_auc_score(ytest.astype(float), ypred))
    mean +=roc_auc_score(ytest.astype(float), ypred)
    mean_list.append(roc_auc_score(ytest.astype(float), ypred))

print("Mean accuracy score: ", mean/n_split)    
print("Confidence Interval: ", get_standard_diviation(mean_list) )

### Validation

In [ ]:
reg = linear_model.Lasso(0.01)
reg = reg.fit(X, y)
ypred = reg.predict(X_val)

print("Accuracy Score: ", roc_auc_score(y_val.astype(float), ypred))

## Lasso t-statistics

In [ ]:
# Get corefficients
params = reg_las.coef_

# Add row to constants
newX = pd.DataFrame({"Constant":np.ones(len(Xtest))}).join(pd.DataFrame(Xtest))
column_names =["incept"]+list(df_both.columns[:-1])

# #calculate std 
m = np.mean(newX, axis=0)
n = len(newX)
std = np.sqrt(np.sum((newX-m)**2, axis=0)/(n))

#Add incepts
params = np.append(reg_las.intercept_,params)

t_values = params/std

# set se of incept to 0
std[0] = 0
std=np.array(std)
t_values[0] = 0
t_values = np.array(t_values)

myDF3 = pd.DataFrame()
myDF3["Coefficients"]= params
myDF3["Standard Errors"] =std
myDF3["t values"] = t_values
names=["incept"]+list(df_both.columns[:-1])
myDF3=myDF3.set_index(pd.Index(names))
myDF3

In [ ]:
#PlAY
k = 7

importance = myDF3["t values"][1:]
n=int((len(importance)-k)/3)
# Get variables, connect to weights (#Note remove id and labels)
variables =[i[:-4] for i in myDF3.index[1:]]

# plot histograms
plt.figure(figsize=(30, 10))
plt.bar([x+k for x in range(len(importance[k+1:n+k]))], importance[k+1:n+k], alpha=0.5, color="b",label="V1",edgecolor=None)
plt.bar([x+k for x in range(len(importance[n+k+1:2*n+k]))], importance[n+k+1:2*n+k], alpha=0.5, color="g",label="V2",edgecolor=None)
plt.bar([x+k for x in range(len(importance[2*n+k:]))], importance[2*n+k:], alpha=0.5, color="r",label="V3",edgecolor=None)

plt.bar([x for x in range(len(importance[1:k-1]))], importance[1:k-1], color="black",label="Static Variables",edgecolor=None)

plt.plot([0,2,10,k+n], [0,0,0,0], c="black")
plt.xticks([x for x in range(len(importance[:n+k]))], variables[:n+k], rotation =90)

#plt.yscale('symlog')
plt.grid(True, linestyle='-.')
#plt.ylim([-0.005,0.006])
plt.tick_params(labelcolor='r', labelsize=20, width=3)
plt.savefig("allfeat")
plt.legend(loc="best",fontsize="xx-large")
plt.show()